# Describe a PIV Result File

In this example, we want to describe the result file of the ILA Vortex pair (source: https://www.pivtec.com/download/samples/VortexPairSeq.zip)

Where to start? Let's first evaluate what we have:

General context:
- The general concept of a dataset is described by `dcat:Dataset`
- The file is described by `pivmeta:PivResultDistribution` and is part of the `dcat:Dataset`

Specific information:<br>
Of greater interest is the PIV process icluding the PIV parameters leading to the dataset. A `dcat:Dataset` is the output of the PIV process. For this, we can use [`m4i:output of`](http://purl.obolibrary.org/obo/RO_0002353):

In [1]:
from pivmetalib import pivmeta, dcat, m4i, schema, sd, prov

In [2]:
dist = pivmeta.PivResultDistribution(
    title='Result File',
    download_URL='file:///vp1a.dat'
)
# the "downloaded" file must exist:
assert dist.download().exists()

In [3]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=dist
)

Before defining the PIV processing steps, we need to describe the software used:

In [4]:
pivtec = pivmeta.PIVSoftware(
    author=prov.Organization(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)
pivtec

PIVSoftware(author=name='PIVTEC GmbH' mbox='info@pivtec.com', has_documentation=https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf)

## Processing steps

A `PivProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [5]:
from pivmetalib.namespace import PIVMETA  # Or define a standard name table here...

In [6]:
pre = pivmeta.ImageManipulation(
    name='Image pre processing',
    realizes_method=[
        pivmeta.ImageRotation(
            description='Rotates the input image by 180 deg',
            has_parameter=m4i.Variable(
                name='rotation',
                value=180,
                has_unit='deg',
                kind_of_quantity='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        pivmeta.ImageFilter(
            description='Applies a median filter to the image',
            has_parameter=pivmeta.Variable(
                standard_name=PIVMETA.image_filter_kernel_size,
                value=3,
            )
        )
    ]
)
pre

C:\Users\da4323\AppData\Local\Temp\ipykernel_25704\1630200954.py:18: UserWarning: Code: image_filter_kernel_size is not defined in namespace PIVMETA
  standard_name=PIVMETA.image_filter_kernel_size,


ImageManipulation(name=Image pre processing, realizes_method=[ImageRotation(description=Rotates the input image by 180 deg, has_parameter=name='rotation' value=180 description=None has_unit='deg' has_kind_of_quantity=None has_symbol=None), ImageFilter(description=Applies a median filter to the image, has_parameter=name=None value=3 description=None has_unit=None has_kind_of_quantity=None has_symbol=None standard_name=Url('https://matthiasprobst.github.io/pivmeta#image_filter_kernel_size'))])

## 2. PIV evaluation

In this step, the PIV algorithm is executed. The evaluation step is characterized by
- the correlation algorithm (e.g. FFT or ...)
- The interrogation method (e.g. multi-pass or multi-grid)

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [7]:
calgo = pivmeta.CorrelationAlgorithm(
    name='Standard (FFT) Correlation',
    description='The default mode of cross-correlation using FFTs to speed the computation. ' 
    'In principle the sum of pixel-wise multiplication of intensities is computed for each ' 
    'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
calgo

CorrelationAlgorithm(name=Standard (FFT) Correlation, description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007).)

### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [8]:
int_meth = pivmeta.InterrogationMethod.Multigrid(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    correlation_algorithm=calgo,
    has_parameter=[
        m4i.Variable(
            standard_name=PIVMETA.x_initial_interrogation_window_size,
            value=64,
        ),
        m4i.Variable(
            standard_name=PIVMETA.y_initial_interrogation_window_size,
            value=64,
        ),
        m4i.Variable(
            standard_name=PIVMETA.x_final_interrogation_window_size,
            value=16,
        ),
        m4i.Variable(
            standard_name=PIVMETA.y_final_interrogation_window_size,
            value=16,
        ),
        m4i.Variable(
            standard_name=PIVMETA.x_final_interrogation_window_overlap_size,
            value=8,
        ),
        m4i.Variable(
            standard_name=PIVMETA.y_final_interrogation_window_overlap_size,
            value=8,
        ),
        m4i.Variable(
            standard_name=PIVMETA.number_of_multigrid_passes,
            value=3,
        )
    ]
)
int_meth

C:\Users\da4323\AppData\Local\Temp\ipykernel_25704\178228079.py:7: UserWarning: Code: x_initial_interrogation_window_size is not defined in namespace PIVMETA
  standard_name=PIVMETA.x_initial_interrogation_window_size,
C:\Users\da4323\AppData\Local\Temp\ipykernel_25704\178228079.py:11: UserWarning: Code: y_initial_interrogation_window_size is not defined in namespace PIVMETA
  standard_name=PIVMETA.y_initial_interrogation_window_size,
C:\Users\da4323\AppData\Local\Temp\ipykernel_25704\178228079.py:31: UserWarning: Code: number_of_multigrid_passes is not defined in namespace PIVMETA
  standard_name=PIVMETA.number_of_multigrid_passes,


Multigrid(name=Multigrid interrogation method, description=Run a multigrid PIV algorithm on all images, has_parameter=[Variable(value=64), Variable(value=64), Variable(value=16), Variable(value=16), Variable(value=8), Variable(value=8), Variable(value=3)], correlation_algorithm=name='Standard (FFT) Correlation' description='The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007).' has_parameter=None)

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [9]:
median_test = m4i.Method(
    name='normalized median test',
    has_parameter=m4i.Variable(
        name='threshold',
        value=3.0
    )
)

In [10]:
dyn_mean = m4i.Method(
    name='dynmaic mean test',
    has_parameter=[
        m4i.Variable(
            name='mean',
            value=2.0
        ),
        m4i.Variable(
            name='var',
            value=1.0
        )
    ]
)

In [11]:
proc = pivmeta.PivProcessingStep(
    name='piv evaluation',
    realizes_method=[
        int_meth,
        median_test,
        dyn_mean
    ]
)
proc

PivProcessingStep(name=piv evaluation, realizes_method=[Multigrid(name=Multigrid interrogation method, description=Run a multigrid PIV algorithm on all images, has_parameter=[Variable(value=64), Variable(value=64), Variable(value=16), Variable(value=16), Variable(value=8), Variable(value=8), Variable(value=3)], correlation_algorithm=name='Standard (FFT) Correlation' description='The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007).' has_parameter=None), Method(name=normalized median test, has_parameter=name='threshold' value=3.0 description=None has_unit=None has_kind_of_quantity=None has_symbol=None), Method(name=dynmaic mean test, has_parameter=[Variable(name=mean, value=2.0), Variable(name=var, value=1.0)])])

In [12]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    has_parameter=m4i.Variable(name='kernel', value=2.0)
)

In [13]:
post = pivmeta.PostProcessingStep(
    name='Post processing',
    realizes_method=data_smoothing
)
            

AttributeError: module 'pivmetalib.pivmeta' has no attribute 'PostProcessingStep'

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [ ]:
post.has_output = ds

## 4. dump piv run to JSON-LD

In [ ]:
piv = m4i.ProcessingStep(
    name='PIV Run',
    starts_with=pre,
    ends_with=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = pivtec
proc.has_employed_tool = pivtec
post.has_employed_tool = pivtec

In [ ]:
with open('piv_process.json', 'w') as f:
    f.write(proc.dump_jsonld())

In [ ]:
from pivmetalib.jsonld import dump_jsonld

In [ ]:
print(dump_jsonld([pre, proc]))

# Query JSON-LD file

t.b.d